# Music Emotion Recognizer

`feature_selector` -> function that implements different types of feature selection:
* Pearson correlation
* Backward elimination
* RFE (Recursive Feature Elimination)
* embedded method
* RreliefF algorithm

`MER_regression` -> function that return the best regressor in RMSE sense. Arguments of this function are:

* `static` = True, False, use static or dynamic data
* `data_type` = 'audio', 'eda', 'fusion', on which type of data want to have the regression
* `feature_sel` = True, False, apply or not feature selection
* `target` = 'Valence(mean)', 'Valence(std)', 'Arousal(mean)', 'Arousal(std)'
* `selection_method` = 'Pearson', 'backward', 'RFE', 'embedded', 'RreliefF'
* `pearson_treshold` = [0-1] value
* `max_features` = max number of features for RreliefF method

MER regression is computed on different data:
* audio
* EDA
* fusion, which means audio+EDA

Each data is divided into:
* static, evaluated the entire data
* dynamic, evaluated with windows of 1s with 50% overlap


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from math import sqrt

import IPython.display as ipd
import statsmodels.api as sm

from sklearn.linear_model import Lasso, ElasticNet, Ridge, LinearRegression, RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import RFE
from sklearn.model_selection import cross_validate, train_test_split

from skrebate import ReliefF

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
def load_audio_dataset(data):
    features = data[data.columns[:-1]].values
    label1 = data[data.columns[-2]].values
    label2 = data[data.columns[-1]].values
    labels = label1+label2
    #scaler = StandardScaler(copy=False)
    #scaler.fit_transform(features)
    return features, labels

In [4]:
def rmse(y, y_pred):
    return sqrt(mean_squared_error(y, y_pred))

def r2(y, y_pred):
    return r2_score(y, y_pred)

regressors = {
    'LR': LinearRegression(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    'Ridge': Ridge(),
    'kNN': KNeighborsRegressor(),
    'SVRrbf': SVR(kernel='rbf', gamma='scale'),
    'SVRpoly': SVR(kernel='poly', gamma='scale'),
    'SVRlinear': SVR(kernel='linear', gamma='scale'),
    'DT': DecisionTreeRegressor(max_depth=5),
    'RF': RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1),
}

In [5]:
def cross_val_regression_RMSE(regressors, features, labels, preprocessfunc):
    columns = list(regressors.keys())
    scores = pd.DataFrame(columns=columns, index=['RMSE'])

    for reg_name, reg in regressors.items(): #for reg_name, reg in tqdm(regressors.items(), desc='regressors'):
        scorer = {'rmse': make_scorer(rmse)}
        reg = make_pipeline(*preprocessfunc, reg)
        reg_score = cross_validate(reg, features, labels, scoring=scorer, cv=10, return_train_score=False) 
        scores.loc['RMSE', reg_name] = reg_score['test_rmse'].mean()
        #scores.loc['R', reg_name] = reg_score['test_r'].mean()

    #mean_rmse = scores.mean(axis=1)
    #std_rmse = scores.std(axis=1)
    
    #scores['Mean'] = mean_rmse
    #scores['std'] = std_rmse
    return scores

def cross_val_regression_r2(regressors, features, labels, preprocessfunc):
    columns = list(regressors.keys())
    scores_r2 = pd.DataFrame(columns=columns, index=['R2'])

    for reg_name, reg in regressors.items():
        scorer_r2 = {'r2': make_scorer(r2)}
        reg = make_pipeline(*preprocessfunc, reg)
        reg_score_r2 = cross_validate(reg, features, labels, scoring=scorer_r2, cv=10, return_train_score=False) 
        scores_r2.loc['R2', reg_name] = reg_score_r2['test_r2'].mean()

    return scores_r2

def format_scores(scores):
    def highlight(s):
        is_min = s == min(s)
#         is_max = s == max(s)
#         is_max_or_min = (is_min | is_max)
        return ['background-color: yellow' if v else '' for v in is_min]
    scores = scores.style.apply(highlight, axis=1, subset=pd.IndexSlice[:, :scores.columns[-2]])
    return scores.format('{:.3f}')

In [6]:
# target = 'Valence(mean)', 'Valence(std)', 'Arousal(mean)', 'Arousal(std)'
# selection_method = 'Pearson', 'backward', 'RFE', 'embedded', 'RreliefF'
# pearson_treshold = [0-1] value
# max_features = max number of features for RreliefF method

def feature_selector(df, selection_method, pearson_treshold=0, max_features=0):
    
    a = list(df.columns.values)
    target = a[-1]
    X = df.drop(columns=[target]) # feature matrix
    y = df[target] # target variable
    
    if selection_method == 'Pearson':
        
        cor = df.corr()
        cor_target = abs(cor[target])
        relevant_features = cor_target[cor_target>pearson_treshold]
        pearson_list = relevant_features.axes
        pearson_df = df[np.intersect1d(df.columns, pearson_list)]
        pearson_df = pearson_df.drop([target], axis=1)
        pearson_df = pearson_df.join(y)
        
        return pearson_df
    
    if selection_method == 'backward':
        
        cols = list(X.columns)
        pmax = 1
        while (len(cols)>0):
            p = []
            X_1 = X[cols]
            X_1 = sm.add_constant(X_1)
            model = sm.OLS(y,X_1).fit()
            p = pd.Series(model.pvalues.values[1:],index = cols)      
            pmax = max(p)
            feature_with_p_max = p.idxmax()
            if(pmax>0.05):
                cols.remove(feature_with_p_max)
            else:
                break
        selected_features_BE = cols
        backward_df = df[np.intersect1d(df.columns, selected_features_BE)]
        backward_df = backward_df.join(y)
        
        return backward_df
    
    if selection_method == 'RFE':
        
        nof_list=np.arange(1,X.shape[1]) 
        high_score=0
        nof=0           
        score_list =[]
        for n in range(len(nof_list)):
            X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
            model = LinearRegression()
            rfe = RFE(model,nof_list[n])
            X_train_rfe = rfe.fit_transform(X_train,y_train)
            X_test_rfe = rfe.transform(X_test)
            model.fit(X_train_rfe,y_train)
            score = model.score(X_test_rfe,y_test)
            score_list.append(score)
            if(score>high_score):
                high_score = score
                nof = nof_list[n]

        cols = list(X.columns)
        model = LinearRegression()
        #Initializing RFE model
        rfe = RFE(model, nof)             
        #Transforming data using RFE
        X_rfe = rfe.fit_transform(X,y)  
        #Fitting the data to model
        model.fit(X_rfe,y)              
        temp = pd.Series(rfe.support_,index = cols)
        selected_features_rfe = temp[temp==True].index        
        rfe_df = df[np.intersect1d(df.columns, selected_features_rfe)]
        rfe_df = rfe_df.join(y)
        
        return rfe_df
    
    if selection_method == 'embedded':
        
        reg = LassoCV()
        reg.fit(X, y)
        coef = pd.Series(reg.coef_, index = X.columns)
        imp_coef = coef.sort_values()
        embedded_list = imp_coef[imp_coef!=0].axes
        embedded_df = df[np.intersect1d(df.columns, embedded_list)]
        embedded_df = embedded_df.join(y)
        
        return embedded_df
    
    if selection_method == 'RreliefF':
        
        features, labels = X.values, y.values
        clf = make_pipeline(ReliefF(n_features_to_select=2, n_neighbors=100),RandomForestClassifier(n_estimators=100))
        X_train, X_test, y_train, y_test = train_test_split(features, labels)
        fs = ReliefF()
        fs.fit(X_train, y_train)

        features_array = fs.top_features_
        best = features_array[:max_features]
        RreliefF_df = df.iloc[:,best]
        RreliefF_df = RreliefF_df.join(y)
        
        return RreliefF_df
    

In [20]:
# static = True, False, use the static or dynamic data
# data_type = 'audio', 'eda', 'fusion', on which type of data want to have the regression
# feature_sel = True, False, apply or not feature selection
# target = 'Valence(mean)', 'Valence(std)', 'Arousal(mean)', 'Arousal(std)'
# selection_method = 'Pearson', 'backward', 'RFE', 'embedded', 'RreliefF'
# pearson_treshold = [0-1] value
# max_features = max number of features for RreliefF method


def MER_regression(features_dir, VA_dir, dynamic, data_type, feature_sel,
                   selection_method='', pearson_treshold=0, max_features=0):

    if dynamic == False:
        VA_mean_dir = VA_dir + '/static_annotations.csv'
        VA_std_dir = VA_dir + '/static_annotations_std.csv'
        sd = 'static'
        lab = ['music_ID']
    if dynamic == True:
        VA_mean_dir = VA_dir + '/dynamic_annotations.csv'
        VA_std_dir = VA_dir + '/dynamic_annotations_std.csv'
        sd = 'dynamic'
        lab = ['music_ID','frame']
        #lab1 = 'music_ID', 'frame'

    VA_mean_df = pd.read_csv(VA_mean_dir)
    VA_std_df = pd.read_csv(VA_std_dir)
    
    if data_type == 'audio':
        if dynamic == False:
            data_dir = features_dir + '/static_features.csv'
        if dynamic == True:
            data_dir = features_dir + '/dynamic_features.csv'
        data_df = pd.read_csv(data_dir)
        data_VA_df = pd.merge(data_df, VA_mean_df, on=lab)
        data_VA_df = pd.merge(data_VA_df, VA_std_df, on=lab)
        
    if data_type == 'eda':
        if dynamic == False:
            data_dir = features_dir + '/static_features_EDA.csv'
        if dynamic == True:
            data_dir = features_dir + '/dynamic_features_EDA.csv'
        data_df = pd.read_csv(data_dir)
        data_df = data_df.drop(columns=['subject_ID'])
        data_df = data_df.groupby(by=['music_ID'], as_index=False).mean() # mean over 10 subjects
        data_VA_df = pd.merge(data_df, VA_mean_df, on=lab)
        data_VA_df = pd.merge(data_VA_df, VA_std_df, on=lab)

    if data_type == 'fusion':
        if dynamic == False:
            audio_dir = features_dir + '/static_features.csv'
            eda_dir = features_dir + '/static_features_EDA.csv'
        if dynamic == True:
            audio_dir = features_dir + '/dynamic_features.csv'
            eda_dir = features_dir + '/dynamic_features_EDA.csv'
        audio_df = pd.read_csv(audio_dir)
        audio_VA_df = pd.merge(audio_df, VA_mean_df, on=['music_ID'])
        audio_VA_df = pd.merge(audio_VA_df, VA_std_df, on=['music_ID'])
        eda_df = pd.read_csv(eda_dir)
        eda_df = eda_df.drop(columns=['subject_ID'])
        eda_df = eda_df.groupby(by=['music_ID'], as_index=False).mean() # mean over 10 subjects
        eda_VA_df = pd.merge(eda_df, VA_mean_df, on=lab)
        eda_VA_df = pd.merge(eda_VA_df, VA_std_df, on=lab)
        
        data_VA_df = pd.merge(audio_df, eda_df, on=['music_ID'])
        data_VA_df = pd.merge(data_VA_df, VA_mean_df, on=['music_ID'])
        data_VA_df = pd.merge(data_VA_df, VA_std_df, on=['music_ID'])
    
    data_VA_df = data_VA_df.dropna(axis='columns')
    
    data_va_mean = data_VA_df.drop(columns=lab)
    data_va_mean = data_va_mean.drop(columns=['Arousal(std)', 'Valence(std)'])
    data_va_std = data_VA_df.drop(columns=lab)
    data_va_std = data_va_std.drop(columns=['Arousal(mean)', 'Valence(mean)'])
    
    #if feature_sel == False:
        #print(sd, str(data_type),'features with no feature selection:\n')
    
    if feature_sel == True:
        data_va_mean = feature_selector(data_va_mean, selection_method, pearson_treshold, max_features)
        data_va_std = feature_selector(data_va_std, selection_method, pearson_treshold, max_features)
        #print(sd, str(data_type),'features with', selection_method, 'feature selection:\n')
       
    prefunc = [StandardScaler()]
    
    #print('In VA (mean) dimension...')
    features_va_mean, labels_va_mean = load_audio_dataset(data_va_mean)
    scores_va_mean_RMSE = cross_val_regression_RMSE(regressors, features_va_mean, labels_va_mean, prefunc)
    #print(scores_va_mean_RMSE)
    scores_va_mean_r2 = cross_val_regression_r2(regressors, features_va_mean, labels_va_mean, prefunc)
    #print(scores_va_mean_r2)
    #ipd.display(format_scores(scores_va_mean))

    #print('In VA (std) dimension...')
    features_va_std, labels_va_std = load_audio_dataset(data_va_std)
    scores_va_std_RMSE = cross_val_regression_RMSE(regressors, features_va_std, labels_va_std, prefunc)
    scores_va_std_r2 = cross_val_regression_r2(regressors, features_va_std, labels_va_std, prefunc)
    #ipd.display(format_scores(scores_va_std))

    return dynamic, data_type, feature_sel, selection_method, scores_va_mean_RMSE, scores_va_mean_r2, scores_va_std_RMSE, scores_va_std_r2, pearson_treshold, max_features


In [8]:
f_dir = '/Users/gioelepozzi/Desktop/data/features_thesis'
VA_dir = '/Users/gioelepozzi/Desktop/data/annotations_thesis'

# Static

In [9]:
MER_static_list = []


In [10]:
# static audio

MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'audio', False, selection_method='none'))
print('1 audio')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='Pearson', pearson_treshold=0.1))
print('2 audio')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='backward'))
print('3 audio')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='RFE'))
print('4 audio')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='embedded'))
print('5 audio')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='RreliefF', max_features=10))


# static eda

MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'eda', False, selection_method='none'))
print('1 eda')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='Pearson', pearson_treshold=0.05))
print('2 eda')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='backward'))
print('3 eda')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='RFE'))
print('4 eda')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='embedded'))
print('5 eda')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='RreliefF', max_features=10))


# static fusion

MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'fusion', False, selection_method='none'))
print('1 fusion')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'fusion', feature_sel=True, selection_method='Pearson', pearson_treshold=0.1))
print('2 fusion')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'fusion', feature_sel=True, selection_method='backward'))
print('3 fusion')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'fusion', feature_sel=True, selection_method='RFE'))
print('4 fusion')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'fusion', feature_sel=True, selection_method='embedded'))
print('5 fusion')
MER_static_list.append(MER_regression(f_dir, VA_dir, False, 'fusion', feature_sel=True, selection_method='RreliefF', max_features=10))



1 audio
2 audio
3 audio
4 audio
5 audio
1 eda
2 eda
3 eda
4 eda
5 eda
1 fusion
2 fusion
3 fusion
4 fusion
5 fusion


In [13]:
data = []

for k in range(18):
    a = MER_static_list[k]
    dynamic = a[0]
    data_type = a[1]
    feature_sel = a[2]
    selection_method = a[3]
    scores_va_mean_RMSE = a[4]
    scores_va_mean_r2 = a[5]
    scores_va_std_RMSE = a[6]
    scores_va_std_r2 = a[7]
    pearson_treshold = a[8]
    max_features = a[9]

    regressor_va_mean_RMSE, regressor_va_mean_r2, rmse_va_mean, r2_va_mean = [], [], [], []
    regressor_va_std_RMSE, regressor_va_std_r2, rmse_va_std, r2_va_std = [], [], [], []
    
    for i in range(0,10):
        regressor_va_mean_RMSE.append(scores_va_mean_RMSE.columns[i])
        regressor_va_mean_r2.append(scores_va_mean_r2.columns[i])
        rmse_va_mean.append(scores_va_mean_RMSE[scores_va_mean_RMSE.columns[i]][0])
        r2_va_mean.append(scores_va_mean_r2[scores_va_mean_r2.columns[i]][0])

        regressor_va_std_RMSE.append(scores_va_std_RMSE.columns[i])
        regressor_va_std_r2.append(scores_va_std_r2.columns[i])
        rmse_va_std.append(scores_va_std_RMSE[scores_va_std_RMSE.columns[i]][0])
        r2_va_std.append(scores_va_std_r2[scores_va_std_r2.columns[i]][0])
    
    for j in range(len(regressor_va_mean_RMSE)):
        data.append([k, regressor_va_mean_RMSE[j], 'VA(mean)', rmse_va_mean[j], r2_va_mean[j],
                     data_type, dynamic, feature_sel, selection_method, pearson_treshold, max_features])

    for j in range(len(regressor_va_std_RMSE)):
        data.append([k, regressor_va_std_RMSE[j], 'VA(std)', rmse_va_std[j], r2_va_std[j],
                     data_type, dynamic, feature_sel, selection_method, pearson_treshold, max_features])



In [14]:
df = pd.DataFrame(data, columns = ['k','regressor', 'VA', 'RMSE','r2', 'file_type', 'dynamic', 'feature_sel',
                                   'selection_method', 'pearson_treshold', 'max_features'])


In [15]:
df_ordered_RMSE = df.sort_values(by='RMSE', ascending=True)
df_ordered_r2 = df.sort_values(by='r2', ascending=False)
#worst_RMSE = df_ordered[ df_ordered['RMSE'] > 0.2 ].index
#df_ordered.drop(worst_RMSE , inplace=True)
#df_ordered.drop(df_ordered , inplace=True)

In [16]:
df_ordered_RMSE.head()

,k,regressor,VA,RMSE,r2,file_type,dynamic,feature_sel,selection_method,pearson_treshold,max_features
290,14,LR,VA(std),0.042466,0.939422,fusion,False,True,backward,0.0,0
50,2,LR,VA(std),0.042806,0.938098,audio,False,True,backward,0.0,0
293,14,Ridge,VA(std),0.042894,0.938457,fusion,False,True,backward,0.0,0
53,2,Ridge,VA(std),0.043178,0.937169,audio,False,True,backward,0.0,0
110,5,LR,VA(std),0.044254,0.948504,audio,False,True,RreliefF,0.0,10


In [17]:
df_ordered_r2.head()

,k,regressor,VA,RMSE,r2,file_type,dynamic,feature_sel,selection_method,pearson_treshold,max_features
170,8,LR,VA(std),0.044285,0.960619,eda,False,True,backward,0.0,0
333,16,Ridge,VA(std),0.044319,0.960193,fusion,False,True,embedded,0.0,0
330,16,LR,VA(std),0.044324,0.960174,fusion,False,True,embedded,0.0,0
173,8,Ridge,VA(std),0.044567,0.960142,eda,False,True,backward,0.0,0
210,10,LR,VA(std),0.044698,0.959805,eda,False,True,embedded,0.0,0


In [19]:
df_ordered_RMSE.to_csv('df_ordered_RMSE.csv', index=False)
df_ordered_r2.to_csv('df_ordered_r2.csv', index=False)

# Dynamic

In [ ]:
MER_dynamic_list = []

In [ ]:
# dynamic audio

MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'audio', False, selection_method='none'))
print('1 audio')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='Pearson', pearson_treshold=0.1))
print('2 audio')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='backward'))
print('3 audio')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='RFE'))
print('4 audio')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='embedded'))
print('5 audio')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='RreliefF', max_features=10))

# dynamic eda

MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'eda', False, selection_method='none'))
print('1 eda')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='Pearson', pearson_treshold=0.05))
print('2 eda')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='backward'))
print('3 eda')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='RFE'))
print('4 eda')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='embedded'))
print('5 eda')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='RreliefF', max_features=10))

# dynamic fusion

MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'fusion', False, selection_method='none'))
print('1 fusion')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='Pearson', pearson_treshold=0.1))
print('2 fusion')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='backward'))
print('3 fusion')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='RFE'))
print('4 fusion')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='embedded'))
print('5 fusion')
MER_dynamic_list.append(MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='RreliefF', max_features=10))


In [ ]:
data_dynamic = []

for k in range(18):
    a = MER_dynamic_list[k]
    dynamic = a[0]
    data_type = a[1]
    feature_sel = a[2]
    selection_method = a[3]
    scores_va_mean_RMSE = a[4]
    scores_va_mean_r2 = a[5]
    scores_va_std_RMSE = a[6]
    scores_va_std_r2 = a[7]
    pearson_treshold = a[8]
    max_features = a[9]

    regressor_va_mean_RMSE, regressor_va_mean_r2, rmse_va_mean, r2_va_mean = [], [], [], []
    regressor_va_std_RMSE, regressor_va_std_r2, rmse_va_std, r2_va_std = [], [], [], []
    
    for i in range(0,10):
        regressor_va_mean_RMSE.append(scores_va_mean_RMSE.columns[i])
        regressor_va_mean_r2.append(scores_va_mean_r2.columns[i])
        rmse_va_mean.append(scores_va_mean_RMSE[scores_va_mean_RMSE.columns[i]][0])
        r2_va_mean.append(scores_va_mean_r2[scores_va_mean_r2.columns[i]][0])

        regressor_va_std_RMSE.append(scores_va_std_RMSE.columns[i])
        regressor_va_std_r2.append(scores_va_std_r2.columns[i])
        rmse_va_std.append(scores_va_std_RMSE[scores_va_std_RMSE.columns[i]][0])
        r2_va_std.append(scores_va_std_r2[scores_va_std_r2.columns[i]][0])
    
    for j in range(len(regressor_va_mean)):
        data.append([k, regressor_va_mean[j], 'VA(mean)', rmse_va_mean[j], r2_va_mean[j], data_type, dynamic, feature_sel,
                     selection_method, pearson_treshold, max_features])

    for j in range(len(regressor_va_std)):
        data.append([k, regressor_va_std[j], 'VA(std)', rmse_va_std[j], r2_va_std[j], data_type, dynamic, feature_sel,
                     selection_method, pearson_treshold, max_features])




In [ ]:
# Create the pandas DataFrame 

df_dynamic = pd.DataFrame(data, columns = ['k','regressor', 'VA', 'RMSE','r2', 'file_type', 'dynamic',
                                           'feature_sel', 'selection_method', 'pearson_treshold', 'max_features'])


In [ ]:
df_dynamic_ordered_RMSE = df_dynamic.sort_values(by='RMSE', ascending=True)
df_dynamic_ordered_r2 = df_dynamic.sort_values(by='r2', ascending=False)
#worst_RMSE = df_ordered[ df_ordered['RMSE'] > 0.2 ].index
#df_ordered.drop(worst_RMSE , inplace=True)
#df_ordered.drop(df_ordered , inplace=True)

In [ ]:
# Static data

MER_regression(f_dir, VA_dir, False, 'audio', False, selection_method='none')
MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='Pearson', pearson_treshold=0.1)
MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='backward')
MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='RFE')
MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='embedded')
MER_regression(f_dir, VA_dir, False, 'audio', feature_sel=True, selection_method='RreliefF', max_features=10)

MER_regression(f_dir, VA_dir, False, 'eda', False)
MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='Pearson', pearson_treshold=0.05)
MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='backward')
MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='RFE')
MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='embedded')
MER_regression(f_dir, VA_dir, False, 'eda', feature_sel=True, selection_method='RreliefF', max_features=10)

MER_regression(f_dir, VA_dir, False, 'fusion', False)
MER_regression(f_dir, VA_dir, False, 'fusion', feature_sel=True, selection_method='Pearson', pearson_treshold=0.1)
MER_regression(f_dir, VA_dir, False, 'fusion', feature_sel=True, selection_method='backward')
MER_regression(f_dir, VA_dir, False, 'fusion', feature_sel=True, selection_method='RFE')
MER_regression(f_dir, VA_dir, False, 'fusion', feature_sel=True, selection_method='embedded')
MER_regression(f_dir, VA_dir, False, 'fusion', feature_se=True, selection_method='RreliefF', max_features=10)

In [ ]:
# Dynamic data

MER_regression(f_dir, VA_dir, True, 'audio', False, selection_method='none')
MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='Pearson', pearson_treshold=0.1)
MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='backward')
MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='RFE')
MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='embedded')
MER_regression(f_dir, VA_dir, True, 'audio', feature_sel=True, selection_method='RreliefF', max_features=10)

MER_regression(f_dir, VA_dir, True, 'eda', False, selection_method='none')
MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='Pearson', pearson_treshold=0.05)
MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='backward')
MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='RFE')
MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='embedded')
MER_regression(f_dir, VA_dir, True, 'eda', feature_sel=True, selection_method='RreliefF', max_features=10)

MER_regression(f_dir, VA_dir, True, 'fusion', False, selection_method='none')
MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='Pearson', pearson_treshold=0.1)
MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='backward')
MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='RFE')
MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='embedded')
MER_regression(f_dir, VA_dir, True, 'fusion', feature_sel=True, selection_method='RreliefF', max_features=10)
